### 📌 LangGraph 개요
- **LangChain 기반의 Workflow Orchestration 라이브러리**
- LLM + Tool + Memory + 조건부 분기 등을 **그래프 구조**로 표현
- 핵심 개념:
  - **State**: 노드 간 공유되는 상태 (TypedDict 등으로 정의)
  - **Node**: 상태를 입력받아 가공하는 함수/LLM 호출/Tool
  - **Edge**: 노드 간 연결 (조건부 분기 가능)
  - **Compile**: 그래프를 실행 가능한 객체로 변환
- 장점:
  - 체인보다 **복잡한 워크플로우**를 유연하게 구성
  - 디버깅/재실행/분기 제어가 용이


In [3]:
# # 설치: pip install langgraph

# from langgraph.graph import StateGraph, END

# # 간단한 Hello World 그래프 예시
# def hello_node(state):
#     print("Hello, LangGraph!")
#     return state

# # 그래프 정의
# graph = StateGraph(dict)
# graph.add_node("hello", hello_node)
# graph.set_entry_point("hello")
# graph.set_finish_point("hello")

# app = graph.compile()
# app.invoke({})

### 📌 LangGraph 기본 구조
- **State**: TypedDict로 명시적 타입 지정
- **Node**: 상태를 받아서 가공 후 반환
- **Edge**: 단순 연결 or 조건부 연결
- **Compile**: `graph.compile()` 후 `.invoke()`로 실행

In [4]:
from typing import TypedDict

# 상태 정의
class MyState(TypedDict):
    question: str
    answer: str

# 노드 함수
def answer_node(state: MyState):
    state["answer"] = f"답변: {state['question']}에 대한 예시 응답"
    return state

# 그래프 빌드
graph = StateGraph(MyState)
graph.add_node("answer", answer_node)
graph.set_entry_point("answer")
graph.set_finish_point("answer")

app = graph.compile()
print(app.invoke({"question": "LangGraph란?"}))

{'question': 'LangGraph란?', 'answer': '답변: LangGraph란?에 대한 예시 응답'}


### 📌 자주 쓰는 문법
- **TypedDict**: 상태 타입 정의
- **Annotated**: 상태 업데이트 방식 지정 (append 등)
- **Reducer**: 여러 노드 결과를 누적 관리


In [5]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph

# Reducer: 상태 업데이트 방법 정의
class State(TypedDict):
    history: Annotated[list[str], "append"]

def add_message(state: State):
    state["history"].append("Hello!")
    return state

graph = StateGraph(State)
graph.add_node("add", add_message)
graph.set_entry_point("add")
graph.set_finish_point("add")

app = graph.compile()
print(app.invoke({"history": []}))

{'history': ['Hello!']}


### 📌 챗봇 구현
- 상태 = `messages` (대화 이력)
- 노드 = LLM 호출
- 실행 = HumanMessage 입력 → AIMessage 응답


In [7]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [8]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

llm = ChatOpenAI(model="gpt-3.5-turbo")

class ChatState(TypedDict):
    messages: list

def llm_node(state: ChatState):
    response = llm.invoke(state["messages"])
    state["messages"].append(response)
    return state

graph = StateGraph(ChatState)
graph.add_node("chat", llm_node)
graph.set_entry_point("chat")
graph.set_finish_point("chat")

app = graph.compile()
print(app.invoke({"messages": [HumanMessage(content="안녕 LangGraph?")]}))

{'messages': [HumanMessage(content='안녕 LangGraph?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C9w8JsXb3su4KODfLMrRtun6kwv84', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--11490e46-0296-43bc-97c7-4244ed7c4c4d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


### 📌 Function Calling + Conditional Edge
- LLM의 함수 호출(Function Calling)과 도구 호출을 노드로 구성
- 조건에 따라 다른 노드로 이동 (if-else 구조를 Edge로 표현)

In [11]:
# pip install langgraph

from typing import TypedDict
from langgraph.graph import StateGraph, END

# 1) 상태 정의
class MyState(TypedDict):
    question: str
    answer: str

# 2) 라우팅 함수 (분기 결정)
def decide_route(state: MyState) -> str:
    return "weather" if "날씨" in state["question"] else "general"

# 3) 작업 노드
def weather_node(state: MyState):
    state["answer"] = "오늘은 맑습니다."
    return state

def general_node(state: MyState):
    state["answer"] = "일반적인 질문 답변입니다."
    return state

# 4) 그래프 구성
graph = StateGraph(MyState)

# (1) 라우터 노드: 상태만 통과시키는 no-op
def router(state: MyState): 
    return state

graph.add_node("router", router)
graph.add_node("weather", weather_node)
graph.add_node("general", general_node)

# (2) 라우터에서 조건부 분기
graph.add_conditional_edges(
    "router",
    decide_route,                 # 분기 함수: "weather" 또는 "general" 반환
    {"weather": "weather", "general": "general"}  # 반환값 → 노드 매핑
)

# (3) 종료 엣지
graph.add_edge("weather", END)
graph.add_edge("general", END)

# (4) 진입 지점
graph.set_entry_point("router")

# 5) 컴파일 & 실행
app = graph.compile()
print(app.invoke({"question": "오늘 날씨 어때?"}))
print(app.invoke({"question": "LangGraph가 뭐야?"}))

{'question': '오늘 날씨 어때?', 'answer': '오늘은 맑습니다.'}
{'question': 'LangGraph가 뭐야?', 'answer': '일반적인 질문 답변입니다.'}


### 📌 Checkpointer (메모리)
- LangGraph는 **Checkpointer**로 실행 상태 저장
- `thread_id` 별로 대화 맥락 유지 가능
- MemorySaver = 기본 메모리형 저장소


In [13]:
# pip install langgraph langchain-openai

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import AnyMessage, add_messages

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 1) 상태에 'messages'를 정의하고, 누적 방식을 add_messages로 지정
class ChatState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 2) LLM 노드: 전체 messages를 넣어 답변 받고, 새 메시지로 반환 (리듀서가 자동 append)
def chat_node(state: ChatState):
    ai_msg = llm.invoke(state["messages"])
    return {"messages": [ai_msg]}  # ← 반환을 리스트로! (리듀서가 기존에 append함)

# 3) 그래프 구성
graph = StateGraph(ChatState)
graph.add_node("chat", chat_node)
graph.add_edge(START, "chat")
graph.add_edge("chat", END)

# 4) 체크포인터 연결
checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)

# 5) 동일 thread_id로 두 번 호출 → 두 번째 호출에서 과거 대화가 자동 주입
cfg = {"configurable": {"thread_id": "1"}}

r1 = app.invoke({"messages": [HumanMessage(content="너 이름이 뭐야?")]}, config=cfg)
print("1차 응답:", r1["messages"][-1].content)

r2 = app.invoke({"messages": [HumanMessage(content="내가 방금 뭐라고 했지?")]}, config=cfg)
print("2차 응답:", r2["messages"][-1].content)

1차 응답: 저는 AI 어시스턴트이며, 이름은 없습니다. 어떻게 도와드릴까요?
2차 응답: 당신은 "너 이름이 뭐야?"라고 물었습니다.


### 📌 Stream 모드 & Interrupt
- `.stream()` → 노드별 중간 결과 확인
- **Interrupt** → 특정 지점에서 실행 중단, 사람이 개입

In [15]:
# 예: ChatState(messages=...) 패턴 + MemorySaver 사용 중
cfg = {"configurable": {"thread_id": "session-1"}}  # 세션 식별자

# 기본 스트림 (각 노드의 출력 묶음 단위)
for update in app.stream({"messages": [{"type":"human","content":"LangGraph란?"}]}, config=cfg):
    print("중간 업데이트:", update)

중간 업데이트: {'chat': {'messages': [AIMessage(content='LangGraph는 언어 간 상호 작용을 시각적으로 나타내는 그래프 형식의 도구입니다. 이 도구는 다양한 언어 간의 관계를 시각적으로 이해하고 분석하는 데 도움을 줄 수 있습니다. LangGraph를 사용하면 언어 간의 공통점과 차이점을 쉽게 파악할 수 있으며, 언어 간의 상호 작용을 보다 효과적으로 이해할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 12, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C9wCmL2xe5GTCjPjTrEIpJFUijODR', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b8c10ee7-f056-4e22-ba5d-a4057a8c4ec4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 147, 'total_tokens': 159, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 📌 Naive RAG
- **전통적 구조**:
  1. Retriever로 문서 검색
  2. Generator로 LLM 응답 생성
- LangGraph는 이 흐름을 **시각적으로, 모듈적으로 구성** 가능


In [20]:
# pip install langgraph langchain-openai langchain-community faiss-cpu

from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 1) 데이터 & 검색기 준비 -------------------------------------------------------
texts = [
    "LangGraph는 그래프 기반 LLM 워크플로우 오케스트레이션 라이브러리다.",
    "RAG는 검색(Retrieval)과 생성(Generation)을 결합한 접근법이다.",
    "LangChain은 LLM 앱 개발을 쉽게 하는 프레임워크다.",
]
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(texts, emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})  # ← NameError 해결 포인트

# 2) LLM 준비 -------------------------------------------------------------------
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 3) 상태 정의 ------------------------------------------------------------------
class RAGState(TypedDict):
    question: str
    context: str
    answer: str

# 4) 노드 정의 ------------------------------------------------------------------
def retrieve_node(state: RAGState) -> RAGState:
    docs = retriever.get_relevant_documents(state["question"])
    ctx = "\n".join(d.page_content for d in docs)
    state["context"] = ctx
    return state

def generate_node(state: RAGState) -> RAGState:
    prompt = f"""당신은 한국어 도우미입니다.
질문: {state['question']}
문맥:
{state['context']}

문맥을 근거로 간결하게 답변하세요."""
    state["answer"] = llm.invoke(prompt).content
    return state

# 5) 그래프 구성 ----------------------------------------------------------------
graph = StateGraph(RAGState)
graph.add_node("retrieve", retrieve_node)
graph.add_node("generate", generate_node)
graph.add_edge(START, "retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", END)

app = graph.compile()

# 6) 실행 -----------------------------------------------------------------------
out = app.invoke({"question": "LangGraph란?"})
print("== 답변 ==")
print(out["answer"])

C:\Users\james\AppData\Local\Temp\ipykernel_5840\1378141614.py:29: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(state["question"])


== 답변 ==
LangGraph는 그래프 기반 LLM 워크플로우 오케스트레이션 라이브러리입니다.


### 📌 할루시네이션 평가 모듈
- 응답에 근거 문맥이 포함되지 않으면 경고 추가
- **품질 관리 노드**로 활용


In [22]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# --- 검색/LLM 준비 ---
texts = [
    "LangGraph는 그래프 기반 LLM 워크플로 오케스트레이션 라이브러리다.",
    "RAG는 검색과 생성을 결합한 접근법이다.",
    "LangChain은 LLM 앱 개발을 쉽게 한다.",
]
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(texts, emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# --- 상태 & 노드 정의 ---
class RAGState(TypedDict):
    question: str
    context: str
    answer: str

def retrieve_node(state: RAGState) -> RAGState:
    docs = retriever.get_relevant_documents(state["question"])
    state["context"] = "\n".join(d.page_content for d in docs)
    return state

def generate_node(state: RAGState) -> RAGState:
    prompt = f"""질문: {state['question']}
문맥:
{state['context']}

문맥을 근거로 간결하게 답하세요."""
    state["answer"] = llm.invoke(prompt).content
    return state

def hallucination_check(state: RAGState) -> RAGState:
    # 아주 단순한 데모용 규칙 (실전은 RAGAS/LLM 검증 등 권장)
    if "LangGraph" not in state["answer"]:
        state["answer"] += " (문맥과 무관한 내용일 수 있음)"
    return state

# --- 그래프 구성(compile 전에 전부 추가) ---
graph = StateGraph(RAGState)
graph.add_node("retrieve", retrieve_node)
graph.add_node("generate", generate_node)
graph.add_node("check", hallucination_check)

graph.add_edge(START, "retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", "check")
graph.add_edge("check", END)  # ← 종료는 END로 두는 편이 최신 패턴에 맞음

# 반드시 여기서 컴파일
app = graph.compile()

# 실행
out = app.invoke({"question": "LangGraph란?"})
print(out["answer"])

LangGraph는 그래프 기반 LLM 워크플로 오케스트레이션 라이브러리이다.


### 📌 웹 검색 노드
- Retriever 외부에 **실시간 검색 노드** 추가 가능
- 최신 정보 기반 RAG 구현


In [23]:
# 필요시 설치: pip install duckduckgo-search langgraph langchain-openai langchain-community faiss-cpu
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.tools import DuckDuckGoSearchRun

# ----- 0) 리소스 준비 -----
texts = [
    "LangGraph는 그래프 기반 LLM 워크플로 오케스트레이션 라이브러리다.",
    "RAG는 검색과 생성을 결합한 접근법이다.",
    "LangChain은 LLM 앱 개발을 쉽게 한다.",
]
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(texts, emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
ddg = DuckDuckGoSearchRun()

# ----- 1) 상태 정의 -----
class RAGState(TypedDict):
    question: str
    context: str
    answer: str

# ----- 2) 노드 정의 -----
def rewrite_node(state: RAGState) -> RAGState:
    # (옵션) 쿼리 재작성: 필요 없으면 이 노드/엣지 제거해도 됩니다.
    new_q = llm.invoke(f"다음 질문을 웹/문서 검색에 적합하게 한 문장으로 재작성: {state['question']}").content
    state["question"] = new_q.strip()
    return state

def websearch_node(state: RAGState) -> RAGState:
    # DuckDuckGo 웹검색 결과 문자열(요약)을 context에 추가
    result = ddg.run(state["question"])
    state["context"] = (state.get("context") or "") + ("\n" if state.get("context") else "") + result
    return state

def retrieve_node(state: RAGState) -> RAGState:
    docs = retriever.get_relevant_documents(state["question"])
    ctx = "\n".join(d.page_content for d in docs)
    state["context"] = (state.get("context") + "\n" if state.get("context") else "") + ctx
    return state

def generate_node(state: RAGState) -> RAGState:
    prompt = f"""당신은 한국어 도우미입니다.
질문: {state['question']}
문맥:
{state['context']}

문맥을 근거로 간결하고 사실적으로 답하세요."""
    state["answer"] = llm.invoke(prompt).content
    return state

def hallucination_check(state: RAGState) -> RAGState:
    # 데모용 간단 검증
    if "LangGraph" not in state["answer"] and "LangGraph" in state["context"]:
        state["answer"] += " (문맥과 무관할 수 있음)"
    return state

# ----- 3) 그래프 구성 -----
graph = StateGraph(RAGState)
graph.add_node("rewrite", rewrite_node)          # (옵션)
graph.add_node("websearch", websearch_node)      # 새 노드
graph.add_node("retrieve", retrieve_node)
graph.add_node("generate", generate_node)
graph.add_node("check", hallucination_check)

# 흐름: START → (rewrite) → websearch → retrieve → generate → check → END
graph.add_edge(START, "rewrite")
graph.add_edge("rewrite", "websearch")
graph.add_edge("websearch", "retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", "check")
graph.add_edge("check", END)

# ----- 4) 반드시 여기서 컴파일! -----
app = graph.compile()

# ----- 5) 실행 -----
out = app.invoke({"question": "LangGraph란?"})
print(out["answer"])

LangGraph는 언어 간 상호 연결성을 시각적으로 보여주는 그래프 시각화 도구입니다. LangGraph는 그래프 기반 LLM 워크플로 오케스트레이션 라이브러리이며, LangChain은 LLM 앱 개발을 쉽게 도와주는 도구입니다. RAG는 검색과 생성을 결합한 접근법을 사용하는 도구입니다.


### 📌 쿼리 재작성 노드
- 원본 질문을 검색 친화적으로 재작성
- 검색 Recall 향상 → RAG 성능 개선


In [24]:
# pip install langgraph langchain-openai langchain-community faiss-cpu duckduckgo-search
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.tools import DuckDuckGoSearchRun

# 리소스
texts = [
    "LangGraph는 그래프 기반 LLM 워크플로 오케스트레이션 라이브러리다.",
    "RAG는 검색과 생성을 결합한 접근법이다.",
    "LangChain은 LLM 앱 개발을 쉽게 한다.",
]
emb = OpenAIEmbeddings(model="text-embedding-3-small")
vs = FAISS.from_texts(texts, emb)
retriever = vs.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
ddg = DuckDuckGoSearchRun()

# 상태
class RAGState(TypedDict):
    question: str
    context: str
    answer: str

# 노드
def rewrite_query(state: RAGState) -> RAGState:
    state["question"] = llm.invoke(
        f"다음 질문을 검색에 최적화된 쿼리로 다시 써줘: {state['question']}"
    ).content.strip()
    return state

def websearch_node(state: RAGState) -> RAGState:
    result = ddg.run(state["question"])
    state["context"] = (state.get("context") + "\n" if state.get("context") else "") + result
    return state

def retrieve_node(state: RAGState) -> RAGState:
    docs = retriever.get_relevant_documents(state["question"])
    ctx = "\n".join(d.page_content for d in docs)
    state["context"] = (state.get("context") + "\n" if state.get("context") else "") + ctx
    return state

def generate_node(state: RAGState) -> RAGState:
    prompt = f"""당신은 한국어 도우미입니다.
질문: {state['question']}
문맥:
{state['context']}

문맥을 근거로 간결하고 사실적으로 답하세요."""
    state["answer"] = llm.invoke(prompt).content
    return state

def hallucination_check(state: RAGState) -> RAGState:
    if "LangGraph" not in state["answer"] and "LangGraph" in state.get("context", ""):
        state["answer"] += " (문맥과 무관할 수 있음)"
    return state

# 그래프 구성 (모두 추가 → 마지막에 컴파일)
graph = StateGraph(RAGState)
graph.add_node("rewrite", rewrite_query)
graph.add_node("websearch", websearch_node)
graph.add_node("retrieve", retrieve_node)
graph.add_node("generate", generate_node)
graph.add_node("check", hallucination_check)

graph.add_edge(START, "rewrite")
graph.add_edge("rewrite", "websearch")
graph.add_edge("websearch", "retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", "check")
graph.add_edge("check", END)

app = graph.compile()  # ← 한 번만 컴파일

# 실행
print(app.invoke({"question": "LangGraph란?"})["answer"])

LangGraph는 Multi-Agent 특화 프레임워크로, 다중 에이전트 시스템을 구축할 수 있게 해주는 도구입니다. 각 에이전트는 특정 목표를 향해 자율적으로 행동하며, 다른 에이전트들과 협업하여 복잡한 문제를 해결할 수 있습니다. LangGraph는 AI의 사고 흐름을 그릴 수 있는 도구로, 복잡한 AI 시스템을 쉽게 만들어주는 역할을 합니다.


## ✅ 최종 정리

- LangGraph 개요: State, Node, Edge, Compile → 체인보다 강력한 워크플로우 엔진

- 핵심 기능: TypedDict, Annotated, Reducer / Memory(Checkpointer) / Stream / Interrupt / Replay

- 구조 설계: Naive RAG → 평가 모듈 → 웹검색 → 쿼리 재작성 등 모듈 단위 확장 가능

- 👉 LangGraph는 단순 체인에서 한 단계 더 나아가, 복잡한 RAG/에이전트 시스템을 엔지니어링하는 필수 도구